In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyCustomCNN(nn.Module):
    def __init__(self, num_classes=39):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)
        self.fc1   = nn.Linear(64 * 56 * 56, 128)
        self.fc2   = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)                # 112 → 56
        x = F.relu(self.conv2(x))
        x = self.pool(x)                # 56 → 28
        x = x.view(x.size(0), -1) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [2]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import torch.optim as optim


data_dir    = os.path.join(os.getcwd(), "Plant_leave_diseases_dataset_with_augmentation")
num_classes = 39
batch_size  = 32
device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k_folds     = 5
num_epochs  = 15
base_lr     = 1e-3



transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
full_ds = datasets.ImageFolder(root=data_dir, transform=transform)
labels  = full_ds.targets
skf     = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

fold_accs = []
for fold, (train_idx, val_idx) in enumerate(skf.split(range(len(labels)), labels), 1):
    print(f"--- Fold {fold}/{k_folds} ---")
    train_ds = Subset(full_ds, train_idx)
    val_ds   = Subset(full_ds, val_idx)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4)
    
    model = MyCustomCNN(num_classes=num_classes).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=base_lr)
    criterion = nn.CrossEntropyLoss()
    

    for epoch in range(1, num_epochs+1):

        model.train()
        total_loss = 0
        for imgs, targets in train_loader:
            imgs, targets = imgs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss    = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * imgs.size(0)
        train_loss = total_loss / len(train_ds)
        
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for imgs, targets in val_loader:
                imgs = imgs.to(device)
                logits = model(imgs)
                preds  = logits.argmax(dim=1).cpu().tolist()
                all_preds.extend(preds)
                all_labels.extend(targets.tolist())
        val_acc = accuracy_score(all_labels, all_preds)
        
        print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    fold_accs.append(val_acc)
    torch.save(model.state_dict(), f"custom_model_fold{fold}.pth")

print(f"\n{k_folds}-fold CV mean accuracy: {sum(fold_accs)/len(fold_accs):.4f}")


NameError: name 'torch' is not defined

In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import torch.optim as optim


data_dir    = os.path.join(os.getcwd(), "Plant_leave_diseases_dataset_with_augmentation")
num_classes = 39
batch_size  = 32
device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k_folds     = 5
num_epochs  = 15
base_lr     = 1e-3

use_amp = True 

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
full_ds = datasets.ImageFolder(root=data_dir, transform=transform)
labels  = full_ds.targets
skf     = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

fold_accs = []
for fold, (train_idx, val_idx) in enumerate(skf.split(range(len(labels)), labels), 1):
    print(f"--- Fold {fold}/{k_folds} ---")
    train_ds = Subset(full_ds, train_idx)
    val_ds   = Subset(full_ds, val_idx)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4)
    

    model = MyCustomCNN(num_classes=num_classes).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=base_lr)
    criterion = nn.CrossEntropyLoss()
    scaler = torch.amp.GradScaler('cuda', enabled = use_amp)
    
    for epoch in range(1, num_epochs+1):

        model.train()
        total_loss = 0
        for imgs, targets in train_loader:
            imgs, targets = imgs.to(device), targets.to(device)

            if not use_amp:
                optimizer.zero_grad()
                outputs = model(imgs)
                loss    = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
            else:
                optimizer.zero_grad()
                with torch.autocast(device_type=device, dtype=torch.float16):
                    outputs = model(imgs)
                    loss    = criterion(outputs, targets)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

            total_loss += loss.item() * imgs.size(0)
        train_loss = total_loss / len(train_ds)
        
        # 验证
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for imgs, targets in val_loader:
                imgs = imgs.to(device)
                logits = model(imgs)
                preds  = logits.argmax(dim=1).cpu().tolist()
                all_preds.extend(preds)
                all_labels.extend(targets.tolist())
        val_acc = accuracy_score(all_labels, all_preds)
        
        print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    fold_accs.append(val_acc)
    torch.save(model.state_dict(), f"custom_model_fold{fold}.pth")

print(f"\n{k_folds}-fold CV mean accuracy: {sum(fold_accs)/len(fold_accs):.4f}")


NameError: name 'torch' is not defined